In [ ]:
!pip install -U transformers torch sentencepiece accelerate bitsandbytes langchain

TEXT Summarization Excerisze with Falconsai model - Smallts

In [ ]:
from transformers import pipeline
import torch

summarize = pipeline("summarization",
                     model="Falconsai/text_summarization")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
from google.colab import files
import io

uploaded = files.upload()
document_content = []
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  content = uploaded[fn].decode('utf-8')
  document_content.append(content)


Saving test_sentences.txt to test_sentences.txt
User uploaded file "test_sentences.txt" with length 12210 bytes


In [27]:
def print_summaries_to_console(text_chunks, all_summaries):
    """
    Prints the original text's chunking information, summaries for each chunk,
    and a consolidated summary to the console.

    Args:
        text_chunks (list): A list of strings, where each string is a chunk of the original text.
        all_summaries (list): A list of strings, where each string is the summary for a corresponding text chunk.
    """
    print(f"Original text split into {len(text_chunks)} chunks for summarization.\n")

    print("--- Summaries per Chunk ---")
    for i, chunk_summary_text in enumerate(all_summaries):
        print(f"\nChunk {i+1}/{len(all_summaries)}:")
        print(chunk_summary_text)

    print("\n--- Consolidated Summary (joining all chunk summaries) ---")
    final_summary = " ".join(all_summaries)
    print(final_summary.replace('. ', '.\n'))


In [28]:
from transformers import AutoTokenizer

'''
Initialize the tokenizer for the model to accurately count tokens.
Since the model has both an encoder and decoder, the model supports no more than 512 tokens.
We need to split the input for larger texts.
'''
tokenizer = AutoTokenizer.from_pretrained("Falconsai/text_summarization")

def split_text_by_tokens(text, tokenizer, max_chunk_tokens=None, overlap_tokens=50):
    """
    Splits a long text into chunks of token_ids, then decodes them back to text.
    This is crucial for models with a maximum sequence length, like the summarization model.
    """
    if max_chunk_tokens is None:
        '''
        Use the model's max length and provide a buffer
        '''
        max_chunk_tokens = tokenizer.model_max_length - 50
    token_ids = tokenizer.encode(text, add_special_tokens=False)

    chunks = []
    current_start_idx = 0

    while current_start_idx < len(token_ids):
        current_end_idx = min(current_start_idx + max_chunk_tokens, len(token_ids))

        '''
        Decode the tokens back to text for summarization
        '''
        chunk_text = tokenizer.decode(token_ids[current_start_idx:current_end_idx], skip_special_tokens=True)
        chunks.append(chunk_text)

        '''
        Move the start index for the next chunk, accounting for overlap
        '''
        next_start_idx = current_start_idx + (max_chunk_tokens - overlap_tokens)

        '''
        If the next_start_idx would go past the end, break the loop.
        The remaining part will be handled by the logic after the loop.
        '''
        if next_start_idx >= len(token_ids):
            break

        current_start_idx = next_start_idx

    '''
    Handle any remaining tokens that might not have been captured as a full chunk due to overlap calculation.
    This ensures the very end of the text is always included, and avoids duplicate chunks if the last one was already covered.
    '''
    if current_start_idx < len(token_ids):
        remaining_text = tokenizer.decode(token_ids[current_start_idx:], skip_special_tokens=True).strip()
        if remaining_text and (not chunks or remaining_text != chunks[-1].strip()):
            chunks.append(remaining_text)

    '''
    Remove empty chunks that might arise from edge cases
    '''
    chunks = [chunk for chunk in chunks if chunk.strip()]

    return chunks

'''
Check if 'document_content' is defined, if not, try to populate it from 'uploaded'
'''
if 'document_content' not in locals() and 'document_content' not in globals():
    if 'uploaded' in locals() and uploaded:
        '''
        Assuming only one file is uploaded for this context
        '''
        first_fn = list(uploaded.keys())[0]
        document_content = [uploaded[first_fn].decode('utf-8')]
    else:
        raise NameError("document_content is not defined and no uploaded file found to populate it. Please run the file upload cell first.")

'''
Assuming 'document_content' contains the entire document as a single string at index 0.
This was established in a previous step.
'''
full_text = document_content[0]

'''
Split the full text into chunks that fit the model's token limit (e.g., 450 tokens).
This is done to avoid the 'Token indices sequence length is longer' error.
'''
text_chunks = split_text_by_tokens(full_text, tokenizer, max_chunk_tokens=450, overlap_tokens=50)

all_summaries = []
for i, chunk in enumerate(text_chunks):
    '''
    The pipeline expects a list of strings, even for a single input
    '''
    chunk_summary_result = summarize([chunk])
    summary_text = chunk_summary_result[0]['summary_text']
    all_summaries.append(summary_text)

'''
Call the new function to print the summaries
'''
print_summaries_to_console(text_chunks, all_summaries)


Token indices sequence length is longer than the specified maximum sequence length for this model (2461 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 200, but your input_length is only 64. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)


Original text split into 7 chunks for summarization.

--- Summaries per Chunk ---

Chunk 1/7:
On November 18, 2025, Cloudflare’s network experienced significant failures to deliver network traffic for approximately two hours and ten minutes . Nearly three weeks later, our network again failed to serve traffic for 28% of applications behind our network for about 25 minutes. We published detailed post-mortem blog posts following both incidents, but we know that we have more to do to earn back your trust. Today we are sharing details about the work underway at Cloudfleare to prevent outages like these from happening again.

Chunk 2/7:
We expect Cloudflare’s network to be much more resilient . Break glass procedures allow certain individuals to elevate privilege under certain circumstances to perform urgent actions to resolve high severity scenarios . In the moments leading up to each incident we instantaneously deployed a configuration change in our data centers in hundreds of cities arou